In [1]:
import tensorflow as tf
%load_ext tensorboard
import datetime, os

In [2]:
mnist = tf.keras.datasets.fashion_mnist

(x_train_1, y_train), (x_test_1, y_test) = mnist.load_data()
x_train_1, x_test_1 = x_train_1 / 255.0, x_test_1 / 255.0

x_train = x_train_1.reshape(60000,28,28,1)
x_test = x_test_1.reshape(10000,28,28,1)


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import regularizers
from tensorflow.keras import initializers

model = Sequential()
model.add(Conv2D(kernel_initializer=initializers.GlorotNormal(seed=12),  
                 bias_initializer=initializers.GlorotUniform(seed=12),
    filters=32, kernel_size=(3,3), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPool2D(padding='same'))
model.add(Conv2D(kernel_initializer=initializers.he_normal(seed=12),  
                 bias_initializer=initializers.he_uniform(seed=12),
    filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPool2D(padding='same'))
model.add(Flatten())
model.add(Dense(256, activation='relu',
                     kernel_initializer=initializers.lecun_normal(seed=12),  
                     bias_initializer=initializers.lecun_uniform(seed=12)))
model.add(Dropout(0.2))
model.add(Dense(84, activation='relu',
                    kernel_initializer=initializers.RandomNormal(seed=12),  
                    bias_initializer=initializers.RandomUniform(seed=12)))
model.add(Dropout(0.2))
model.add(Dense(10, activation='linear', kernel_initializer=initializers.he_normal(seed=12),  
                 bias_initializer=initializers.he_uniform(seed=12)))


model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               803072    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)              

In [14]:
model.fit(x_train, y_train, batch_size=1000, epochs=2)

model.evaluate(x_test,  y_test, batch_size=1000, verbose=2)

Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 33s 551us/sample - loss: 0.5775 - accuracy: 0.7896
Epoch 2/2
60000/60000 [==============================] - 34s 568us/sample - loss: 0.3498 - accuracy: 0.8744
10000/1 - 2s - loss: 0.3291 - accuracy: 0.8812


[0.32017512500286105, 0.8812]

In [15]:
model.save_weights("C:\\zaleslaw\\home\\models\\dropout\\weights.h5")

# serialize model to JSON
model_json = model.to_json()
with open("C:\\zaleslaw\\home\\models\\dropout\\modelConfig.json", "w") as json_file:
    json_file.write(model_json)